# Ejemplo de conversión de GAMS

Vamos a mostrar un ejemplo de cómo escribir uno de los modelos de GAMS en JuMP.

Para poder aprovechar la explicación en otro punto del curso utilizaremos el ejemplo 207 (mrp2.gms) que se trata de un modelo de planificación.

El original puede consultarse en: https://www.gams.com/latest/gamslib_ml/libhtml/gamslib_mrp2.html

Un modelo de GAMS estructura datos, variables y ecuaciones. Empezaremos por los datos que se utilizan:

```
Set
   PP 'SKU numbers'  / AJ8172, LQ8811, RN0098, NN1100, WN7342 /
   TT 'time buckets' / 1jan01*8jan01  /
   KK 'resources'    / HR-101, MT-402 /;
```

Alias (TT,TTp), (PP,PPp);

* Esto son los conjuntos de datos que a su vez sirven de subíndices en el modelo. En JuMP generalmente se trabaja con números y no con nombres, por lo que crearemos diccionarios que nos permitan leer los nombres una vez tengamos el resultado con números.
* Por otra parte, los subíndices de un conjunto tienen el mismo nombre que el conjunto. Si uno necesita recorrer un mismo conjunto en dos bucles tiene que crear un "alias". Esto en JuMP no es así ya que los subíndices están separados de los conjuntos. En pocas palabras los alias no se usaran, es una característica que en Julia no se necesita.


In [ ]:
PP = Dict(1 => "AJ8172", 2 => "LQ8811", 3 => "RN0098", 4 => "NN1100", 5 => "WN7342")
TT = Dict(1 => "1Jan01", 2 => "2Jan01", 3 => "3Jan01", 4 => "4Jan01", 5 => "5Jan01", 6 => "6Jan01", 7 => "7Jan01", 8 => "8Jan01")
KK = Dict(1 => "HR-101", 2 => "MT-402")

A continuación se introducen un par de tablas con datos:

<b>tabla de requerimientos</b>

```
Table R(PP,PP) 'number of i to make one j'
            AJ8172  LQ8811  RN0098  NN1100  WN7342
   AJ8172
   LQ8811        2
   RN0098        1
   NN1100                1
   WN7342                1                        ;
```

<b>tabla de demanda</b>

```
Table demand(PP,TT) 'external demand for an item in a period'
            1jan01  2jan01  3jan01  4jan01  5jan01  6jan01  7jan01  8jan01
   AJ8172       20      30      10      20      30      20      30      40
   LQ8811
   RN0098
   NN1100
   WN7342                                                                 ;
```


In [ ]:
R=[[0 0 0 0 0]
    [2 0 0 0 0]
    [1 0 0 0 0]
    [0 1 0 0 0]
    [0 1 0 0 0]]

demand=[[20 30 10 20 30 20 30 40]
        [0 0 0 0 0 0 0 0]
        [0 0 0 0 0 0 0 0]
        [0 0 0 0 0 0 0 0]
        [0 0 0 0 0 0 0 0]]

<b>Parámetros de demanda total y nivel de producción</b>

A continuación tenemos dos parámetros que se calculan a partir de los datos. 

```
Parameter
   lev(PP) 'level in the production tree'
   TD(PP)  'total demand extern plus implicit';
```

A continuación el modelo determina el nivel y la demanda total externa más la implícita. Este cálculo debe hacerse utilizando un for de Julia y requiere entender cómo funciona el loop en GAMS (es un tanto lioso, por lo que sería completamente aceptable introducir los valores de forma manual, pero por si alguien quiere hacerlo, hago la traducción aquí.

```
Scalar runlev 'level iteration' / 0 /;

* Root node get level 0, all other get -1
lev(PP)$(sum(PPp,R(PP,PPp))) = -1;
TD(PP)$(lev(PP)  = 0) = sum(TT,demand(PP,TT));
```

Explicaremos con detalle la ecuación 

```
lev(PP)$(sum(PPp,R(PP,PPp))) = -1;
```

El valor de lev(i) tal que (significado del <b>$</b>) la suma para todo i' de R(i,i') no sea nula tiene valor igual a 0 (el contenido de <b>(sum(PPp,R(PP,PPp)))</b> calcula el valor de R).

De la misma manera 

```
TD(PP)$(lev(PP)  = 0) = sum(TT,demand(PP,TT));
```

Indica que TD(i) para aquellos elementos que tengan valor 0 (primer nivel) es igual a un sumatorio de demanda

In [ ]:
lev=zeros(length(PP))
TD=zeros(length(PP))

for i in 1:length(PP)
    suma=0
    for j in 1:length(PP) #equivale a PPp
        suma += R[i,j]
    end
    if suma== 0
        lev[i]=0
    else
        lev[i]=(-1)
    end
end
println(lev)        
for i in 1:length(PP)
    if(lev[i]==0)
        TD[i]=0
        for j in 1:length(TT)
            TD[i] += demand[i,j]
        end
    end
end
println(TD)

runlev=0


El siguiente componente del modelo es un conjunto de órdenes programadas que calculan a la vez el nivel de los productos y la demanda acumulada (TD).

```
loop(PP$(lev(PP) = runlev),
   runlev = runlev + 1;
   lev(PPp)$R(PPp,PP) = runlev;
   TD(PPp)$R(PPp,PP)  = sum(TT,demand(PPp,TT)) + R(PPp,PP)*TD(PP);
);
```

Para entender qué hace esta función podemos consultar: https://www.gams.com/latest/docs/UG_FlowControl.html

Según la explicación miraremos variables del conjunto PP (los productos) tales que el runlev sea igual al valor actual y para cada elemento ejecutaremos tres órdenes:

* incrementar la variable runlev en uno : ``` runlev = runlev + 1;  ``` 
* Asignar el valor lev a : ``` lev(PPp)$R(PPp,PP) = runlev; ``` 
* calcular TD como una suma : ``` TD(PPp)$R(PPp,PP)  = sum(TT,demand(PPp,TT)) + R(PPp,PP)*TD(PP); ``` 

Nótese que este cálculo sólo se hace para poder evaluar el big-M que se usará después

In [ ]:
runlev=0
for i in 1:length(PP)
    if lev[i]==runlev
        runlev += 1
    end
    for PPp in 1:length(PP)
        if R[PPp,i]>0
            lev[PPp]=runlev
        end
    end
    for PPp in 1:length(PP)
        if R[PPp,i]>0
            TD[PPp] = 0
            for j in 1:length(TT)
                TD[PPp] = demand[PPp,j]
            end
            TD[PPp] = TD[PPp] + R[PPp,i]*TD[i]
        end
    end
end
println(lev)
println(TD)

<b>Seguimos con los datos ...</b>

Más datos, plazo de entrega, inventario inicial y tamaño de lote.

```
Parameter
   LT(PP) 'lead time'
   I(PP)  'beginning inventory'
   LS(PP) 'lot size';

Table SKUdata(PP,*)
            LT    LS    I
   AJ8172    2   100   90
   LQ8811    3   400  300
   RN0098    4   100  100
   NN1100    1     1    0
   WN7342   12  1000  900;

LT(PP) = SKUdata(PP,'LT');
LS(PP) = SKUdata(PP,'LS');
I(PP)  = SKUdata(PP, 'I');
```



In [ ]:
LT=[2 3 4 1 2] #cambiado el 12 por un 2, motivo en clase
LS=[100 400 100 1 1000]
BI=[90 300 100 0 900]

<b>Y acabamos con un conjunto de datos que marcan la información de consumo de recursos (de ahí el uso del término MRP-II</b>

```
Table U(PP,KK) 'fraction of resource k needed by one i'
                HR-101       MT-402
   AJ8172   0.00208333  0.000104166
   LQ8811               0.000333333
   RN0098
   NN1100               0.000001000;
```



In [ ]:
U=[[0.00208333 0.000104166]
    [0 0.000333333]
    [0 0]
    [0 0.000001000]
    [0 0]]

<b> Finalmente calculamos el mejor valor para la big-M partiendo de los datos</b>

```
Parameter M(PP) 'big M for equation defprod';
M(PP) = max(TD(PP),LS(PP));
```

In [ ]:
M=zeros(length(PP))
for i in 1:length(PP)
    M[i]=max(TD[i],LS[i])
end
println(M)

## El modelo

Primero hay que cargar la información necesaria para generar y resolver modelos matemáticos en Julia

In [ ]:
using JuMP
using GLPKMathProgInterface

m = Model(solver = GLPKSolverMIP());

En un segundo paso se definen las variables del modelo

```
Binary   Variable d(PP,TT) 'production indicator';
Positive Variable x(PP,TT) 'number of SKUs to produce';
```

In [ ]:
@variable(m, d[1:length(PP),1:length(TT)],Bin)
@variable(m, x[1:length(PP),1:length(TT)] >= 0); # Models x >=0

### Objetivo y restricciones 

El bloque siguiente incluye tanto el objetivo como las restricciones

```
Variable obj;

Equation
   defobj         'objective function'
   defreq(PP,TT)  'material requirement'
   deflot(PP,TT)  'lot size'
   defprod(PP,TT) 'production indicator'
   defcap(TT,KK)  'capacity';

defobj..         obj =e= sum((PP,TT), (card(TT) - ord(TT) + 1)*x(PP,TT));

defreq(PP,TT)..          sum(TTp$(ord(TTp) <= ord(TT) - LT(PP)), x(PP,TTp)) + I(PP)
                     =g= sum(TTp$(ord(TTp) <= ord(TT)), demand(PP,TTp)
                      +  sum(PPp, R(PP,PPp)*x(PPp,TTp)));

deflot(PP,TT)..  x(PP,TT) =g= d(PP,TT)*LS(PP);

defprod(PP,TT).. x(PP,TT) =l= d(PP,TT)*M(PP);

defcap(TT,KK)..  sum(PP, U(PP,KK)*x(PP,TT)) =l= 1;

```

In [ ]:
@objective(m,Min,sum( (length(TT)-tt +1)*x[pp,tt] for pp in 1:length(PP) for tt in 1:length(TT)) );

# calcular requerimiento de materiales
# para cada producto y periodo de tiempo
# lo que he fabricado tiene que ser suficiente para cubrir las necesidades                
for pp in 1:length(PP)
    for tt in 1:length(TT)
        @constraint(m, sum(x[pp,ttp] for ttp in 1:tt-LT[pp]) + BI[pp]  >= 
            sum(demand[pp,ttp] + sum(R[pp,ppp]*x[ppp,ttp] for ppp in 1:length(PP)) for ttp in 1:tt))
    end
end
# fijar condiciones de lotificación                    
# para cada producto y periodo de tiempo, 
# si se produce (d[pp,tt]) la producción (x[pp,tt]) no puede ser menor que el lote mínimo (LS[pp])
for pp in 1:length(PP)
    for tt in 1:length(TT)
        @constraint(m,x[pp,tt] >= d[pp,tt]*LS[pp])
    end
end

# fijar indicación de producción
# para cada producto y periodo de tiempo,
# si no se produce (binaria d[pp,tt]) la producción tiene que ser 0,
# y si se produce puede ser cualquier número (big M)
for pp in 1:length(PP)
    for tt in 1:length(TT)
        @constraint(m,x[pp,tt] <= d[pp,tt]*M[pp])
    end
end

# fijar límites de capacidad
# para cada periodo y recurso
# el uso de recurso por todos los productos no puede ser mayor al disponible (1)
for tt in 1:length(TT)
    for kk in 1:length(KK)
        @constraint(m,sum(U[pp,kk]*x[pp,tt] for pp in 1:length(PP)) <= 1)
    end
end        

### Definición de componentes de diversos modelos

Podemos resolver distintos modelos usando subconjuntos de restricciones. En JuMP hubiéramos construido diferentes modelos para cada una de las agrupaciones indicadas

```
Model
   mrp   / defobj, defreq, deflot, defprod         /
   mrp2  / defobj, defreq,                  defcap /
   mrp2l / defobj, defreq, deflot, defprod, defcap /;

option optCr = 0.0;

solve mrp   minimizing obj using mip;
solve mrp2  minimizing obj using lp;
solve mrp2l minimizing obj using mip;
```

In [ ]:
#escribimos el modelo
print(m)

#resolvemos
status = solve(m)

In [ ]:
println("**** Status: ",status," Objective value: ", getobjectivevalue(m))
println(d)
println(getvalue(d))
println(x)
println(getvalue(x))

In [ ]:
for i in 1:length(PP)
    for t in 1:length(TT)
        println("d[",i,",",t,"]=",getValue(d[i,t]))
    end
end
for i in 1:length(PP)
    for t in 1:length(TT)
        println("x[",i,",",t,"]=",getValue(x[i,t]))
    end
end

In [ ]:
for i in 1:length(PP)
    for t in 1:length(TT)
        if getValue(d[i,t])>0.01
            println("Se hace un pedido del Producto ",PP[i]," en el periodo ",TT[t]," por ",getValue(x[i,t])," unidades")
        end
    end
end